In [1]:
import piexif
import os.path
from arcgis.geometry import Point, SpatialReference
from arcgis.features import SpatialDataFrame
from uuid import uuid4

def dms_to_dd(dms):
    return dms[0][0] / dms[0][1] + dms[1][0] / dms[1][1] / 60 + dms[2][0] / dms[2][1] / 3600

def get_photo_location(path_to_photo):

    # create a dictionary to populate
    photo = {}

    # get all the exif data
    exif = piexif.load(path_to_photo)

    # if gps data is in the exif data
    if 'GPS' in exif:

        # save the gps data to a variable for easier processing
        gps = exif['GPS']

        # get the x value
        y = dms_to_dd(gps[2])

        # check to ensure the x value is logical, between the north and south pole in degrees
        if y > 100:
            y = None
        else:
            
            # set positive or negative based on hemisphere
            y = -y if gps[1] == b'S' else y

        # get the y value
        x = dms_to_dd(gps[4])
        
        # check to ensure the y value is logical, between the prime merian and the international date line
        if x > 190:
            x = None
        else:
            
            # set positive or negateive based on hemisphere
            x = -x if gps[3] == b'W' else x

        # if both x and y are valid, populate dictionary
        if x and y:
            photo['x'] = x
            photo['y'] = y
            
        else:
            photo['x'] = None
            photo['y'] = None

    else:
        photo['x'] = None
        photo['y'] = None
        
    # return the result
    return photo

def get_photo_point_geometry(path_to_photo):
    
    # get the location where the photos were taken
    photo_location = get_photo_location(path_to_photo)
    
    # if a location is returned, create a point geometry and return it
    if photo_location['x'] and photo_location['y']:
        return Point(photo_location, sr=SpatialReference(wkid=4326))
    else:
        return None
    
def get_photo_spatial_data_frame(path_to_directory_containing_photos):
    
    # use SHAPE as convention for the name of the geometry field
    geometry_field = 'SHAPE'
    
    # create a list of JPG photos contained in the directory as a dictionary
    photo_list = [
        {
            'name': file.split('.')[0], 
            'path': os.path.join(path_to_directory_containing_photos, file),
            'uuid': uuid4().hex
        } 
        for file in os.listdir(path_to_directory_containing_photos) 
        if file.lower().endswith('.jpg')
    ]
    
    # create a spatial data frame from the list of photos
    sdf_photo = SpatialDataFrame(photo_list)
    
    # create the geometry points, and save them in a new geometry field
    sdf_photo[geometry_field] = sdf_photo.path.apply(lambda file_path: get_photo_point_geometry(file_path))
    
    # set the geometry field property
    sdf_photo.set_geometry(geometry_field, inplace=True, sr=SpatialReference(wkid=4326))
    
    # reindex all the data and return the result
    return sdf_photo.reset_index(drop=True)

In [2]:
path_dir = r"..\resources\test_photos"

In [3]:
sdf_photo = get_photo_spatial_data_frame(path_dir)
sdf_photo

,name,path,uuid,SHAPE
0,IMG_1149,..\resources\test_photos\IMG_1149.JPG,8c5b49b11cfd4e3fb385f236b3fdd8b0,"{'x': -121.69461111111112, 'y': 45.32214444444..."
1,IMG_1741,..\resources\test_photos\IMG_1741.JPG,663bec48dfb14cfda260477af73c566e,"{'x': -121.12847777777777, 'y': 44.41288333333..."
2,IMG_1821,..\resources\test_photos\IMG_1821.JPG,c5ea3bdde6024b58a9bda806635fd4c8,"{'x': -122.42665833333334, 'y': 47.23696111111..."


In [4]:
sdf_photo.to_featureset()

{"fields": [], "features": [{"attributes": {"path": "..\\resources\\test_photos\\IMG_1149.JPG", "name": "IMG_1149", "uuid": "8c5b49b11cfd4e3fb385f236b3fdd8b0"}, "geometry": {"x": -121.69461111111112, "y": 45.32214444444445, "sr": {"wkid": 4326}}}, {"attributes": {"path": "..\\resources\\test_photos\\IMG_1741.JPG", "name": "IMG_1741", "uuid": "663bec48dfb14cfda260477af73c566e"}, "geometry": {"x": -121.12847777777777, "y": 44.41288333333333, "sr": {"wkid": 4326}}}, {"attributes": {"path": "..\\resources\\test_photos\\IMG_1821.JPG", "name": "IMG_1821", "uuid": "c5ea3bdde6024b58a9bda806635fd4c8"}, "geometry": {"x": -122.42665833333334, "y": 47.236961111111114, "sr": {"wkid": 4326}}}], "geometryType": "esriGeometryPoint", "spatialReference": {"wkid": 4326}}